<a href="https://colab.research.google.com/github/arun-arunisto/PyTorch-Tutorial/blob/main/Tutorial_4_BuildingModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building the Neural Network

Neural networks comprise of layers/modules that perform operations on data. The `torch.nn` namespace provides all the building blocks you need to build your own neural network. Every module in PyTorch subclasses the `nn.Module`. A neural network is a module itself that consists of other modules (layers). This nested structure allows for building and managing complex architectures easily.

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

### Getting Device for training

We want to be able to train our model on an accelerator such as CUDA, MPS, MTIA, or XPU. If the current accelerator is available, we will use it. Otherwise, we use the CPU.

In [2]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


### Defining the class

We define our neural network by subclassing `nn.Module`, and initialize the neural network layers in `__init__`. Every nn.Module subclass implements the operations on input data in the forward method.

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(224*224, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of `NeuralNetwork`, and move it to the `device`, and print its structure.

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=50176, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model’s forward, along with some background operations. Do not call `model.forward()` directly!

Calling the model on the input returns a 2-dimensional tensor with dim=0 corresponding to each output of 10 raw predicted values for each class, and dim=1 corresponding to the individual values of each output. We get the prediction probabilities by passing it through an instance of the `nn.Softmax` module.

In [5]:
X = torch.rand(1, 224, 224, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([9])


### Model Layers

Lets break down the model layers for that we're going to take sample 3 images with size 224x224

In [6]:
input_image = torch.rand(3,224,224)
print(input_image.size())

torch.Size([3, 224, 224])


#### nn.Flatten

We initialize the `nn.Flatten` layer to convert each 2D 224x224 image into a contiguous array of 50176 pixel values ( the minibatch dimension (at dim=0) is maintained).

In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 50176])


#### nn.Linear

The linear layer is a module that applies a linear transformation on the input using its stored weights and biases.

In [8]:
layer1 = nn.Linear(in_features=224*224, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


#### nn.ReLU
Non-linear activations are what create the complex mappings between the model’s inputs and outputs. They are applied after linear transformations to introduce nonlinearity, helping neural networks learn a wide variety of phenomena.

In this model, we use `nn.ReLU` between our linear layers, but there’s other activations to introduce non-linearity in your model.

In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.0594, -0.0252,  0.3062, -0.0225, -0.1976, -0.4561, -0.5925,  0.1894,
          0.1973,  0.0876,  0.2457,  0.0842, -0.5951,  0.1253,  0.0203, -0.0183,
          0.3220,  0.0423,  0.7395,  0.0906],
        [ 0.2589,  0.0633,  0.1467, -0.2767, -0.2290, -0.4886, -0.9249, -0.0915,
          0.1244,  0.2264,  0.1228, -0.2846, -0.2574, -0.1271,  0.0802,  0.0091,
          0.2663,  0.3821,  0.4340, -0.1244],
        [ 0.0223, -0.3545,  0.0550, -0.0494, -0.3710, -0.5641, -0.7890,  0.6223,
         -0.2643,  0.0519, -0.1150, -0.0309, -0.4660,  0.0301,  0.0324, -0.2104,
          0.4205,  0.1800,  0.3762,  0.0252]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0594, 0.0000, 0.3062, 0.0000, 0.0000, 0.0000, 0.0000, 0.1894, 0.1973,
         0.0876, 0.2457, 0.0842, 0.0000, 0.1253, 0.0203, 0.0000, 0.3220, 0.0423,
         0.7395, 0.0906],
        [0.2589, 0.0633, 0.1467, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1244,
         0.2264, 0.1228, 0.0000, 0.0000, 0.0000, 0.08

#### nn.Sequential

nn.Sequential is an ordered container of modules. The data is passed through all the modules in the same order as defined. You can use sequential containers to put together a quick network like `seq_modules`.

In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,224,224)
logits = seq_modules(input_image)

#### nn.Softmax

The last linear layer of the neural network returns logits - raw values in [-infty, infty] - which are passed to the nn.Softmax module. The logits are scaled to values [0, 1] representing the model’s predicted probabilities for each class. dim parameter indicates the dimension along which the values must sum to 1.

In [11]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

#### Model Parameters

Many layers inside a neural network are parameterized, i.e. have associated weights and biases that are optimized during training. Subclassing `nn.Module` automatically tracks all fields defined inside your model object, and makes all parameters accessible using your model’s parameters() or named_parameters() methods.

In this example, we iterate over each parameter, and print its size and a preview of its values.

In [12]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=50176, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 50176]) | Values : tensor([[ 1.7149e-03, -4.9026e-04,  8.7106e-05,  ..., -1.3367e-03,
          2.7687e-03, -1.4290e-03],
        [-5.8662e-04, -2.6683e-03, -3.3889e-03,  ..., -2.1790e-03,
         -3.9083e-04,  2.6171e-03]], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0039, -0.0036], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0004,  0.0339, -0.0183,  ...,  0.0114,  0.0230,  0.0366],
        [ 0.0063, -0.0373, -0.0404,  ..., -0.0357, -0.0130, -0.0369]],
      